# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


In [3]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,random_state=1)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_test)


In [8]:
for i,col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col],y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.64      0.47      0.54      1550
          1       0.84      0.92      0.88      4951
          2       0.80      0.08      0.14        53

avg / total       0.79      0.81      0.79      6554

request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5415
          1       0.79      0.47      0.59      1139

avg / total       0.88      0.89      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3815
          1       0.74      0.60      0.66      2739

avg / total       0.75      0.75      0.74      6554

medical_help


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.92      0.99      0.96      6001
          1       0.62      0.11      0.19       553

avg / total       0.90      0.92      0.89      6554

medical_products
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6205
          1       0.78      0.11      0.19       349

avg / total       0.94      0.95      0.93      6554

search_and_rescue
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      6387
          1       0.43      0.06      0.11       167

avg / total       0.96      0.97      0.96      6554

security
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      6442
          1       0.00      0.00      0.00       112

avg / total       0.97      0.98      0.97      6554

military
             precision    recall  f1-score   support

          0       0.97      1.00 

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fefb7909950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
"""clf__estimator__n_estimators
clf__estimator__min_samples_leaf
clf__estimator__max_features
clf__estimator__min_samples_split
tfidf__use_idf
vect__max_df"""
from sklearn.metrics import accuracy_score,make_scorer
def scores(y_t,y_p):
    accuracy=[]
    for i,col in enumerate(y_t):
        accuracy.append(accuracy_score(y_t[col],y_p[:,i]))
    score=np.mean(accuracy)
    return score
scores(y_test,y_pred)

0.94452497202726071

In [12]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'clf__estimator__n_estimators': [10,25],
    'clf__estimator__max_features': ['auto','log2'],
    'clf__estimator__min_samples_split':[2,4],
    'tfidf__use_idf':['True','False'],
    'vect__max_df':[0.8,1.0]
}
scorer = make_scorer(scores)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer,verbose =10,n_jobs=-1)

In [13]:
model = cv.fit(X_train,y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.8 
[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.8, score=0.943961278947547, total=  51.0s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.8 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.8, score=0.9443088190418067, total=  51.0s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.8 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.8, score=0.9437324110805955, total=  50.7s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0 


...

[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, score=0.9445249720272607, total=  51.2s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0 
[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, score=0.9447114569558879, total=  50.3s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0 
[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, score=0.9436561217916116, total=  50.3s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=Tr

[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, score=0.9455887837792019, total=  44.7s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0 
[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, score=0.9453260095615909, total=  44.5s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8 
[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8, score=0.9473349608381653, total= 1.5min
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True

[CV]  clf__estimator__max_features=log2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, score=0.9361289119452072, total= 1.0min
[CV] clf__estimator__max_features=log2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8 
[CV]  clf__estimator__max_features=log2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8, score=0.9401552910860206, total= 2.2min
[CV] clf__estimator__max_features=log2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8 
[CV]  clf__estimator__max_features=log2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8, score=0.9403544908961451, total= 2.2min
[CV] clf__estimator__max_features=log2, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, 

[CV]  clf__estimator__max_features=log2, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8, score=0.9404477333604585, total= 1.8min
[CV] clf__estimator__max_features=log2, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8 
[CV]  clf__estimator__max_features=log2, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8, score=0.9406257417014205, total= 1.8min
[CV] clf__estimator__max_features=log2, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8 
[CV]  clf__estimator__max_features=log2, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__max_df=0.8, score=0.9398628488115826, total= 1.8min
[CV] clf__estimator__max_features=log2, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=25, tfidf__use_idf=True, v

[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 157.2min finished


In [14]:
print(model.best_params_)

{'clf__estimator__max_features': 'auto', 'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 25, 'tfidf__use_idf': 'False', 'vect__max_df': 0.8}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
from sklearn.metrics import precision_score,recall_score,accuracy_score
print(scores(y_test,y_pred))
y_pred = model.predict(X_test)

0.944524972027


In [18]:
print(scores(y_test,y_pred))

0.947686739226


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
model.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 9. Export your model as a pickle file

In [ ]:
import joblib
joblib.dump

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.